In [1]:
import numpy as np
import pandas as pd

from subprocess import check_output
print(check_output(["ls", "/Users/yangqingfeng/data/open_data/kaggle/two-sigma/"]).decode("utf8"))

Kaggle-renthop.torrent
images_sample.zip
log.txt
sample_submission.csv
test.json
train.json



In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

df = pd.read_json("/Users/yangqingfeng/data/open_data/kaggle/two-sigma/train.json")
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,low
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,high
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,low
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,low


In [7]:
# naive featue engineering
df['num_photos'] = df['photos'].apply(len)
df['num_features'] = df['features'].apply(len)
df['num_description_words'] = df['description'].apply(lambda x: len(x.split(" ")))
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day

In [8]:
num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]
X = df[num_feats]
y = df["interest_level"]
X.head()

,bathrooms,bedrooms,latitude,longitude,price,num_photos,num_features,num_description_words,created_year,created_month,created_day
10,1.5,3,40.7145,-73.9425,3000,5,0,95,2016,6,24
10000,1.0,2,40.7947,-73.9667,5465,11,5,9,2016,6,12
100004,1.0,1,40.7388,-74.0018,2850,8,4,94,2016,4,17
100007,1.0,1,40.7539,-73.9677,3275,3,2,80,2016,4,18
100013,1.0,4,40.8241,-73.9493,3350,3,1,68,2016,4,28


In [16]:
y.head()

10        medium
10000        low
100004      high
100007       low
100013       low
Name: interest_level, dtype: object

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [13]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
los = log_loss(y_val, y_val_pred)

In [15]:
los

0.6236702320838478

In [18]:
y_val_pred

array([[0.224  , 0.367  , 0.409  ],
       [0.22   , 0.269  , 0.511  ],
       [0.024  , 0.717  , 0.259  ],
       ...,
       [0.019  , 0.776  , 0.205  ],
       [0.022  , 0.763  , 0.215  ],
       [0.02675, 0.9073 , 0.06595]])

In [19]:
y_train.head()

40276       low
76848    medium
77226    medium
22205       low
54216       low
Name: interest_level, dtype: object

In [20]:
# predict test data
df_t = pd.read_json("/Users/yangqingfeng/data/open_data/kaggle/two-sigma/test.json")
df_t['num_photos'] = df_t['photos'].apply(len)
df_t['num_features'] = df_t['features'].apply(len)
df_t['num_description_words'] = df_t['description'].apply(lambda x: len(x.split(" ")))
df_t["created"] = pd.to_datetime(df_t["created"])
df_t["created_year"] = df_t["created"].dt.year
df_t["created_month"] = df_t["created"].dt.month
df_t["created_day"] = df_t["created"].dt.day
X_t = df_t[num_feats]
y_t = clf.predict_proba(X_t)

In [25]:
X_t.head()

,bathrooms,bedrooms,latitude,longitude,price,num_photos,num_features,num_description_words,created_year,created_month,created_day
0,1.0,1,40.7185,-73.9865,2950,8,6,78,2016,6,11
1,1.0,2,40.7278,-74.0000,2850,3,3,35,2016,6,24
100,1.0,1,40.7306,-73.9890,3758,6,3,333,2016,6,3
1000,1.0,2,40.7109,-73.9571,3300,6,10,204,2016,6,11
100000,2.0,2,40.7650,-73.9845,4900,7,14,174,2016,4,12


In [26]:
y_t

array([[0.117     , 0.535     , 0.348     ],
       [0.198     , 0.353     , 0.449     ],
       [0.055     , 0.817     , 0.128     ],
       ...,
       [0.066     , 0.717     , 0.217     ],
       [0.245     , 0.269     , 0.486     ],
       [0.047     , 0.67366667, 0.27933333]])

In [29]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [37]:
sub = pd.DataFrame()
sub['listing_id'] = df_t['listing_id']
for label in ['high', 'medium', 'low']:
    sub[label] = y_t[:, labels2idx[label]]

sub

,listing_id,high,medium,low
0,7142618,0.117000,0.348000,0.535000
1,7210040,0.198000,0.449000,0.353000
100,7103890,0.055000,0.128000,0.817000
1000,7143442,0.104000,0.254000,0.642000
100000,6860601,0.014000,0.330000,0.656000
100001,6840081,0.002000,0.052000,0.946000
100002,6922337,0.081000,0.651000,0.268000
100003,6913616,0.272633,0.474367,0.253000
100005,6937820,0.070000,0.219000,0.711000
100006,6893933,0.038000,0.212000,0.750000
